In [ ]:
!pip install gurobipy

     |████████████████████████████████| 11.1 MB 11.9 MB/s 


In [ ]:
from gurobipy import *

迭代1 主问题

In [ ]:
MP=Model()
SP_Dual=Model()
y= MP.addVars(5, obj=7, vtype=GRB.BINARY, name='y')
z= MP.addVar(obj=1, vtype=GRB.CONTINUOUS, name='z')
#设置参数 InfUnbdInfo
SP_Dual.Params.InfUnbdInfo = 1
MP.optimize()
for i in range(5):
  print('y[%d] = %d'%(i, y[i].x))
print('z = %d'%z.x)
print("Obj:",MP.objVal)

Restricted license - for non-production use only - expires 2022-01-13
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 0 rows, 6 columns and 0 nonzeros
Model fingerprint: 0x5d8b4ab0
Variable types: 1 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 2 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
y[0] = 0
y[1] = 0
y[2] = 0
y[3] = 0
y[4] = 0
z = 0
Obj: 0.0


迭代1 子问题的对偶问题

In [ ]:

#定义对偶变量
Vdual1 = SP_Dual.addVars(3, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name='Vdual1')
Vdual2 = SP_Dual.addVars(5, lb=-GRB.INFINITY,ub=0, vtype=GRB.CONTINUOUS, name='Vdual2')
#对偶模型约束   
x1 = SP_Dual.addConstr(Vdual1[0] + Vdual2[0] <=1)
x2 = SP_Dual.addConstr(Vdual1[1] + Vdual2[1] <=1)
x3 = SP_Dual.addConstr(Vdual1[2] + Vdual2[2] <=1)
x4 = SP_Dual.addConstr(Vdual1[0] + Vdual1[2] + Vdual2[3] <=1)
x5 = SP_Dual.addConstr(Vdual1[0] + Vdual1[1] + Vdual2[4] <=1)

#设置子问题对偶问题目标
SP_Dual.setObjective(8*Vdual1[0] + 3*Vdual1[1] + 5*Vdual1[2] + \
          8*Vdual2[0]*y[0].x + 3*Vdual2[1]*y[1].x + 5*Vdual2[2]*y[2].x + \
          5*Vdual2[3]*y[3].x + 3*Vdual2[4]*y[4].x, GRB.MAXIMIZE)
SP_Dual.optimize()
#print("Obj:",SP_Dual.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5 rows, 8 columns and 12 nonzeros
Model fingerprint: 0xe5376430
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5 rows and 5 columns
Presolve time: 0.02s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 7 iterations and 0.02 seconds
Unbounded model


In [ ]:
#检验子问题不可行，其对偶问题无界
SP_Dual.status == GRB.Status.UNBOUNDED

True

In [ ]:
ray=SP_Dual.UnbdRay
SP_Dual.UnbdRay

[1.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1.0, -1.0]

迭代2 主问题

In [ ]:
#给主问题增加约束
MP.addConstr(8*ray[0] + 3*ray[1] + 5*ray[2] + \
      8*ray[3]*y[0] + 3*ray[4]*y[1] + 5*ray[5]*y[2] + \
      5*ray[6]*y[3] + 3*ray[7]*y[4]<= 0)
MP.optimize()
for i in range(5):
  print('y[%d] = %d'%(i, y[i].x))
print('z = %d'%z.x)
print("Obj:",MP.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 1 rows, 6 columns and 3 nonzeros
Model fingerprint: 0x7fd50d11
Variable types: 1 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [3e+00, 8e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 8e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R0 by 8.000000000

Found heuristic solution: objective 14.0000000
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 2 available processors)

Solution count 2: 7 14 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%
y[0] = 1
y[1] = 0
y[2] = 0
y[3] =

迭代2 子问题的对偶问题

In [ ]:
Vdual2[0].obj = 8*y[0].x
Vdual2[1].obj = 3*y[1].x
Vdual2[2].obj = 5*y[2].x
Vdual2[3].obj = 5*y[3].x
Vdual2[4].obj = 3*y[4].x
SP_Dual.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5 rows, 8 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   0.000000e+00   8.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Unbounded model


In [ ]:
#检验子问题不可行，其对偶问题无界
SP_Dual.status == GRB.Status.UNBOUNDED

True

In [ ]:
ray=SP_Dual.UnbdRay
SP_Dual.UnbdRay

[0.0, 1.0, 1.0, 0.0, -1.0, -1.0, -1.0, -1.0]

迭代3 主问题

In [ ]:
#给主问题增加约束
MP.addConstr(8*ray[0] + 3*ray[1] + 5*ray[2] + \
      8*ray[3]*y[0] + 3*ray[4]*y[1] + 5*ray[5]*y[2] + \
      5*ray[6]*y[3] + 3*ray[7]*y[4]<= 0)
MP.optimize()
for i in range(5):
  print('y[%d] = %d'%(i, y[i].x))
print('z = %d'%z.x)
print("Obj:",MP.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 2 rows, 6 columns and 7 nonzeros
Model fingerprint: 0x4fed8834
Variable types: 1 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [3e+00, 8e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 8e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R1 by 8.000000000

Found heuristic solution: objective 14.0000000
Presolve removed 2 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 2 available processors)

Solution count 1: 14 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.400000000000e+01, best bound 1.400000000000e+01, gap 0.0000%
y[0] = 0
y[1] = 0
y[2] = 0
y[3] = 1

迭代3 子问题的对偶问题

In [ ]:
Vdual2[0].obj = 8*y[0].x
Vdual2[1].obj = 3*y[1].x
Vdual2[2].obj = 5*y[2].x
Vdual2[3].obj = 5*y[3].x
Vdual2[4].obj = 3*y[4].x
SP_Dual.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5 rows, 8 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  8.000000000e+00


In [ ]:
#检验子问题不可行，其对偶问题无界
SP_Dual.status == GRB.Status.UNBOUNDED

False

In [ ]:
#检验子问题最优解
SP_Dual.status == GRB.Status.OPTIMAL

True

In [ ]:
print("Obj:",SP_Dual.objVal)

Obj: 8.0


迭代4 主问题

In [ ]:
for i in range(3):
  print('Vdual1[%d] = %f'%(i, Vdual1[i].x))
for i in range(5):
  print('Vdual2[%d] = %f'%(i, Vdual2[i].x))

Vdual1[0] = 1.000000
Vdual1[1] = 1.000000
Vdual1[2] = 1.000000
Vdual2[0] = 0.000000
Vdual2[1] = 0.000000
Vdual2[2] = 0.000000
Vdual2[3] = -1.000000
Vdual2[4] = -1.000000


In [ ]:
MP.addConstr(8*Vdual1[0].x + 3*Vdual1[1].x + 5*Vdual1[2].x + \
      8*Vdual2[0].x*y[0] + 3*Vdual2[1].x*y[1] + 5*Vdual2[2].x*y[2] + \
      5*Vdual2[3].x*y[3] + 3*Vdual2[4].x*y[4] <= z)
MP.optimize()
for i in range(5):
  print('y[%d] = %d'%(i, y[i].x))
print('z = %d'%z.x)
print("Obj:",MP.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x9978af72
Variable types: 1 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 2e+01]

MIP start from previous solve produced solution with objective 22 (0.02s)
Loaded MIP start from previous solve with objective 22

Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 2 available processors)

Solution count 1: 22 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.200000000000e+01, best bound 2.200000000000e+01, gap 0.0000%
y[0] = 0
y[1] = 0
y[2] = 0
y[3] = 1
y[4] = 1
z = 8
Obj: 22.0


迭代4 子问题的对偶问题

In [ ]:
Vdual2[0].obj = 8*y[0].x
Vdual2[1].obj = 3*y[1].x
Vdual2[2].obj = 5*y[2].x
Vdual2[3].obj = 5*y[3].x
Vdual2[4].obj = 3*y[4].x
SP_Dual.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5 rows, 8 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  8.000000000e+00


In [ ]:
#检验子问题不可行，其对偶问题无界
SP_Dual.status == GRB.Status.UNBOUNDED

False

In [ ]:
#检验子问题最优解
SP_Dual.status == GRB.Status.OPTIMAL

True

In [ ]:
print("Obj:",SP_Dual.objVal)

Obj: 8.0


最终解输出

In [ ]:
x=[]
x.append(x1.Pi)
x.append(x2.Pi)
x.append(x3.Pi)
x.append(x4.Pi)
x.append(x5.Pi)
x

[0.0, 0.0, 0.0, 5.0, 3.0]

In [ ]:
for i in range(5):
  print('y[%d] = %d'%(i, y[i].x))
for i in range(5):
  print('x[%d] = %d'%(i, x[i]))
print('z = %d'%z.x)
print("Obj:",MP.objVal)

y[0] = 0
y[1] = 0
y[2] = 0
y[3] = 1
y[4] = 1
x[0] = 0
x[1] = 0
x[2] = 0
x[3] = 5
x[4] = 3
z = 8
Obj: 22.0
